# Intro

Stanislaw Sieron and Zachary Pasquarello

Business Question: How does Apple Stock Price change in response to external and internal company factors?

For our project we decided to investigate how Apple Stock Price changes in response to external and internal company factors. When testing our question we decided to focus on several factors, including natural disasters, historic stock price, the federal funds rate, and the USD / EURO conversion rate. We decided to focus on a time span of 23.5 years, spanning back to 01-01-2000, because we wanted a large enough data sample to make sure the LSTM was properly trained. 

We collected our data from a variety of sources, with those being listed below:
Natural Disasters: https://www.kaggle.com/datasets/headsortails/us-natural-disaster-declarations
Historic Stock Price: Yfinance API
Federal Funds Rate: FRED
USD / EURO Conversion: https://www.investing.com/currencies/usd-eur-historical-data

For the most part, we downloaded CSV files that we then uploaded into our github, however, for the Apple Historic Stock Price we simply used an API to fetch the stock price. 

The primary purpose of this notebook is to download all of the data sources and organize them in an easy-to-read format. This involved a number of operations including:
1. Group the data by different categories
2. Renaming various columns
3. Getting rid of Null values
4. Transforming Monthly data into Daily data (funds rate)
5. Changing date to date-time format
6. Getting rid of unnecessary columns
7. Combining all of the data sources into a single exportable DF

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import DataFrame
# Imported all of the necessary libraries

In [221]:
inputDf = pd.read_csv("naturalDisasters.csv")
# Imported our naturalDisasters csv
disasterDf = inputDf[["declaration_date", "id"]].copy()
# Created a new copy with only 2 columns
disasterDf["declaration_date"] = pd.to_datetime(disasterDf["declaration_date"])
disasterDf = disasterDf[disasterDf["declaration_date"] > "2000-01-01"]
disasterDf = disasterDf.drop_duplicates(subset = "declaration_date", keep = "first")
disasterDf = disasterDf.rename(columns = {"declaration_date": "Date"})
disasterDf["Date"] = disasterDf["Date"].dt.date

In [222]:
disasterDf

,Date,id
19683,2000-01-03,33968193-0e2c-425b-8774-bd924f86ba6f
19684,2000-01-04,1c71b322-c6e7-47e0-86aa-4208120bc199
19685,2000-01-05,826d227d-f42d-46e1-958b-1214b7ff2b5a
19686,2000-01-10,6a24cc36-cd99-4d82-b40d-9283b8114e39
19697,2000-01-13,1209c5f1-015d-4560-b43d-575778f52ea0
...,...,...
64022,2023-03-10,2d41d26e-5f2b-4e77-862d-4b7c2edb988f
64065,2023-03-15,ee2e0cdd-5738-4d05-9f38-32f30e9f9963
64074,2023-03-20,25fefe48-c47e-40bd-8aaf-58307ec228e6
64082,2023-03-22,bb7d9f5c-9802-4c03-acd1-1db2f53f6c90


In [223]:
appleStock = yf.Ticker("AAPL")
historicalStock = appleStock.history(period="24y")
allStockDf = pd.DataFrame(historicalStock).reset_index()
allStockDf["Date"] = pd.to_datetime(allStockDf["Date"])
stockDf = allStockDf[["Date", "Close"]]
stockDf = stockDf[stockDf["Date"] > "2000-01-01"]
stockDf["Date"] = stockDf["Date"].dt.date

In [224]:
stockDf

,Date,Close
169,2000-01-03,0.850644
170,2000-01-04,0.778926
171,2000-01-05,0.790324
172,2000-01-06,0.721931
173,2000-01-07,0.756128
...,...,...
6035,2023-04-27,168.410004
6036,2023-04-28,169.679993
6037,2023-05-01,169.589996
6038,2023-05-02,168.539993


In [225]:
fundsRateDf = pd.read_csv("federalFundsRate.csv")
fundsRateDf["DATE"] = pd.to_datetime(fundsRateDf["DATE"])
fundsRateDf = fundsRateDf.rename(columns = {"DATE": "Date", "FEDFUNDS": "FedFundsRate"})
fundsRateDf = fundsRateDf.reset_index()
fundsRateDf["RowNumber"] = fundsRateDf.index + 1
fundsRateDf = fundsRateDf.pivot(index = "Date", columns = "FedFundsRate")
startDate = fundsRateDf.index.min() - pd.DateOffset(day=1)
endDate = fundsRateDf.index.max() - pd.DateOffset(day=31)
dates = pd.date_range(startDate, endDate, freq="D")
dates.name = "Date"
fundsRateDf = fundsRateDf.reindex(dates, method="ffill")
fundsRateDf = fundsRateDf.stack("FedFundsRate")
fundsRateDf = fundsRateDf.sort_index(level=1)
fundsRateDf = fundsRateDf.reset_index()
fundsRateDf = fundsRateDf.sort_values(by="Date")
fundsRateDf = fundsRateDf[["Date", "FedFundsRate"]]
fundsRateDf["Date"] = fundsRateDf["Date"].dt.date

In [226]:
fundsRateDf

,Date,FedFundsRate
8096,2000-01-01,5.45
8097,2000-01-02,5.45
8098,2000-01-03,5.45
8099,2000-01-04,5.45
8100,2000-01-05,5.45
...,...,...
7542,2023-04-26,4.83
7543,2023-04-27,4.83
7544,2023-04-28,4.83
7545,2023-04-29,4.83


In [227]:
currencyConversionDf = pd.read_csv("currencyConversionRate.csv")
currencyConversionDf = currencyConversionDf[["Date", "Price"]]
currencyConversionDf["Date"] = pd.to_datetime(currencyConversionDf["Date"])
currencyConversionDf = currencyConversionDf.sort_values(by = "Date")
currencyConversionDf["Date"] = currencyConversionDf["Date"].dt.date

In [228]:
currencyConversionDf

,Date,Price
4999,2000-01-03,0.9743
4998,2000-01-04,0.9700
4997,2000-01-05,0.9693
4996,2000-01-06,0.9688
4995,2000-01-07,0.9717
...,...,...
4,2019-02-28,0.8794
3,2019-03-01,0.8788
2,2019-03-04,0.8817
1,2019-03-05,0.8842


In [272]:
totalDf = pd.merge(stockDf, disasterDf, how="left")
totalDf = pd.merge(totalDf, fundsRateDf, how="inner")
totalDf = pd.merge(totalDf, currencyConversionDf, how="inner")

In [273]:
totalDf["NaturalDisaster"] = np.where(totalDf["id"].isna(), 0, 1)

In [274]:
totalDf = totalDf[["Close", "Date", "FedFundsRate", "Price", "NaturalDisaster"]]
totalDf

,Close,Date,FedFundsRate,Price,NaturalDisaster
0,0.850644,2000-01-03,5.45,0.9743,1
1,0.778926,2000-01-04,5.45,0.9700,1
2,0.790324,2000-01-05,5.45,0.9693,1
3,0.721931,2000-01-06,5.45,0.9688,0
4,0.756128,2000-01-07,5.45,0.9717,0
...,...,...,...,...,...
4818,41.895260,2019-02-28,2.40,0.8794,0
4819,42.335632,2019-03-01,2.41,0.8788,0
4820,42.548553,2019-03-04,2.41,0.8817,1
4821,42.471127,2019-03-05,2.41,0.8842,1


In [275]:
totalDf.to_csv("finalDataFrame.csv")